# Azure ML v2 — Batch Endpoint **BEZ parallel** (wymuszenie `type: model` przez YAML schema)

Ten notebook **wymusza** tworzenie Batch Deployments jako **Model Batch Deployment** (bez `driver/amlbi_main.py`).

Dlaczego YAML?
- W części środowisk/wersji SDK, tworzenie deploymentu przez obiekt Pythona bywa mapowane do trybu parallel.
- YAML schema `modelBatchDeployment.schema.json` z `type: model` jest najpewniejszym sposobem na **brak parallel runtime**.

Co robi:
1) Tworzy 3 clustery AmlCompute (`min_instances=0`) → **0 kosztu bez jobów**
2) Tworzy 1 Batch Endpoint
3) Generuje 3 pliki YAML deploymentów (`type: model`) dla PL/EN/DE
4) Deployuje je do endpointu
5) Ustawia default deployment (opcjonalnie)
6) (Opcjonalnie) Uruchamia joby kodem


## 0) Instalacja (jeśli potrzebujesz)
Odkomentuj, jeśli pracujesz lokalnie.

In [ ]:
# %pip install -U azure-ai-ml azure-identity


## 1) Konfiguracja
Uzupełnij workspace, env i modele. W prod dawaj konkretne wersje.

In [ ]:
SUBSCRIPTION_ID = "<SUBSCRIPTION_ID>"
RESOURCE_GROUP  = "<RESOURCE_GROUP>"
WORKSPACE_NAME  = "<WORKSPACE_NAME>"

BATCH_ENDPOINT_NAME = "doc-classifier-batch"

ENV_NAME    = "model-x-env"
ENV_VERSION = "5"

MODELS = {
    "pl": {"name": "model-x-pl", "version": "1"},
    "en": {"name": "model-x-en", "version": "1"},
    "de": {"name": "model-x-de", "version": "1"},
}

CODE_DIR = "./src"
SCORING_SCRIPT = "score.py"  # musi mieć init() + run(mini_batch)

# 3 osobne clustery, 0 kosztu bez jobów
COMPUTE_SIZE = "Standard_DS3_v2"
MIN_NODES = 0
MAX_NODES_PER_LANG = {"pl": 4, "en": 4, "de": 4}
IDLE_TIME_BEFORE_SCALE_DOWN = 120

YAML_DIR = "./deploy_yamls"


## 2) MLClient + wersja SDK (debug)

In [ ]:
import azure.ai.ml
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

print("azure-ai-ml version:", azure.ai.ml.__version__)

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=SUBSCRIPTION_ID,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WORKSPACE_NAME,
)

print("Workspace:", ml_client.workspaces.get(WORKSPACE_NAME).name)


## 3) Utwórz 3 AmlCompute clustery (min=0)
Izolacja zasobów per język + 0 kosztu bez jobów.

In [ ]:
from azure.ai.ml.entities import AmlCompute

compute_names = {lang: f"cpu-batch-{lang}" for lang in MODELS.keys()}

for lang, cname in compute_names.items():
    compute = AmlCompute(
        name=cname,
        size=COMPUTE_SIZE,
        min_instances=MIN_NODES,
        max_instances=MAX_NODES_PER_LANG[lang],
        idle_time_before_scale_down=IDLE_TIME_BEFORE_SCALE_DOWN,
    )
    print(f"Creating/updating compute: {cname} (min={MIN_NODES}, max={MAX_NODES_PER_LANG[lang]})")
    ml_client.compute.begin_create_or_update(compute).result()

print("Compute ready:", compute_names)


## 4) Utwórz 1 Batch Endpoint

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

endpoint = BatchEndpoint(
    name=BATCH_ENDPOINT_NAME,
    description="Batch endpoint without parallel runtime (type:model deployments via YAML).",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint).result()
print("Batch endpoint ready:", BATCH_ENDPOINT_NAME)


## 5) Wygeneruj YAML-e deploymentów `type: model` i je zdeployuj
To jest **klucz**: schema `modelBatchDeployment` + `type: model`.

In [ ]:
from pathlib import Path
from azure.ai.ml import load_batch_deployment

Path(YAML_DIR).mkdir(parents=True, exist_ok=True)

deployment_names = {lang: f"deploy-{lang}-model" for lang in MODELS.keys()}
schema_url = "https://azuremlschemas.azureedge.net/latest/modelBatchDeployment.schema.json"

for lang, spec in MODELS.items():
    dep_name = deployment_names[lang]
    yml_path = Path(YAML_DIR) / f"{dep_name}.yml"

    yml = f"""\
$schema: {schema_url}
name: {dep_name}
endpoint_name: {BATCH_ENDPOINT_NAME}
type: model

model: azureml:{spec['name']}:{spec['version']}
environment: azureml:{ENV_NAME}:{ENV_VERSION}

code_configuration:
  code: {CODE_DIR}
  scoring_script: {SCORING_SCRIPT}

compute: azureml:{compute_names[lang]}

resources:
  instance_count: 1
"""

    yml_path.write_text(yml, encoding="utf-8")
    print("Wrote:", str(yml_path))

    dep = load_batch_deployment(str(yml_path))
    print(f"Deploying {dep_name} (type:model) -> compute={compute_names[lang]}")
    ml_client.batch_deployments.begin_create_or_update(dep).result()

print("Deployments created:", deployment_names)


## 6) Ustaw default deployment (opcjonalnie)
UI często odpala job na default dep, jeśli nie wybierzesz innego.

In [ ]:
ep = ml_client.batch_endpoints.get(BATCH_ENDPOINT_NAME)
ep.default_deployment_name = deployment_names["en"]
ml_client.batch_endpoints.begin_create_or_update(ep).result()
print("default_deployment_name =", ep.default_deployment_name)


## 7) Debug: wypisz deployment i sprawdź, że to nie parallel
Jeśli nadal widzisz `amlbi_main.py`, to znaczy, że Studio odpala job na innym (starym) deploymencie lub w trybie parallel z UI.
Wtedy uruchom job **kodem** (pkt 8).

In [ ]:
deps = list(ml_client.batch_deployments.list(endpoint_name=BATCH_ENDPOINT_NAME))
print([d.name for d in deps])

d = ml_client.batch_deployments.get(endpoint_name=BATCH_ENDPOINT_NAME, name=deployment_names["en"])
print("DEP CLASS:", type(d))
print(d)


## 8) (Opcjonalnie) Uruchom job KODEM (pewniej niż UI)
Podmień input/output (folder z plikami JSON, jeśli score.py czyta JSON).

In [ ]:
from azure.ai.ml.entities import BatchJob, Input
import time

INPUTS = {
    "pl": "azureml://datastores/workspaceblobstore/paths/in/pl/",
    "en": "azureml://datastores/workspaceblobstore/paths/in/en/",
    "de": "azureml://datastores/workspaceblobstore/paths/in/de/",
}

OUTPUTS = {
    "pl": "azureml://datastores/workspaceblobstore/paths/out/pl/",
    "en": "azureml://datastores/workspaceblobstore/paths/out/en/",
    "de": "azureml://datastores/workspaceblobstore/paths/out/de/",
}

ts = int(time.time())
submitted = {}

for lang in MODELS.keys():
    job = BatchJob(
        name=f"run-{lang}-{ts}",
        endpoint_name=BATCH_ENDPOINT_NAME,
        deployment_name=deployment_names[lang],
        inputs={"input_data": Input(type="uri_folder", path=INPUTS[lang])},
        outputs={"output_data": Input(type="uri_folder", path=OUTPUTS[lang])},
    )

    created = ml_client.batch_jobs.begin_create_or_update(job).result()
    submitted[lang] = created.name
    print("Submitted:", lang, created.name)

print("Submitted jobs:", submitted)
